# Automated image processing for horizontal channels
## Script description
For the description of an automated image processing procedure to track the moving interface and calculate the contact angles in horizontal channels of a serial sequence of images.

The procedure contains selection of region of interest, wall detection, interface detection and contact angle calculation.

The separate functions are saved in `shared_functions.py`. See the reposity `README.md` for general syntax information.

| Parameters  | Description | Required |
|-------------|-------------|----------|
| `ROI`    | Region of interest: `'upstream'` and `'downstream'` for contact angle calculation, `'x-axis'` for interface movement.        | yes      |
| `direction`    | The direction of ROI: `'horizontal'` .        | yes      |
| `pixel_size`  | Pixel size of image.         | yes      |
| `framerate` | Optical framerate of images.         | yes      |
| `resolution` | Image resolution.         | yes      |
| `n_lines` | The number of horizontal lines for vertical wall edges detection (`n_lines=10`).         | yes      |
| `thres` | Threshold value to classify the pixel values (`thres=50`).         | yes      |
| `poly_degree` | The polynomial degreee of interface fitting with RANSAC (`poly_degree=3`).         | yes      |
| `residual_thres` | Maximum residual for a data sample to be classified as an inlier (`residual_thres=15`).         | yes      |
| `max_trials` | Maximum number of iterations for random sample selection (`max_trials=1000`).         | yes      |
| `min_samples` | Minimum number of samples chosen randomly from original data (`min_samples=1`).         | yes      |
| `n_pixels` | The number of offset-pixels away from channel wall edges for interface tracking (`n_pixels=3`).         | yes      |

In [ ]:
from shared_functions import *

In [ ]:
# define path
path_geometry = "Images/W0.4mm_R0.1mm/"
path_flowrate = "0.004ml_s_200fps/"

path_images = os.path.join(path_geometry, path_flowrate)
path_output = os.path.join(path_geometry, path_flowrate, 'postprocessed')
path_results = os.path.join(path_geometry, path_flowrate, 'results')

if not os.path.isdir(path_output):
        os.mkdir(path_output)
        
if not os.path.isdir(path_results):
        os.mkdir(path_results)
                
image_files = select_images(path_images, start=180, end=185)
image_files.sort()

# Crop the preliminary ROI from the image: contact angle measurement
ROI = 'upstream'
#ROI = 'downstream'

# Crop the preliminary ROI from the image: interface displacement
#ROI = 'x-axis'
# Title for figures
title = str(ROI)

# Direction of ROI
direction = 'horizontal'
# Select the preliminary ROI
x, y, w, h = define_ROI(ROI)

# Number of vertical lines for horizontal channel wall detection
n_lines = 10
# Polynomial degree for interface fitting
poly_degree = 3
# Number of pixels away from the channel wall edges for interface tracking
n_pixels = 3

# Detect the horizontal channel wall y-coordinates on the first image
wall_image = cv2.imread(os.path.join(path_images, image_files[0]))
top_wall_y, bottom_wall_y = detect_walls(wall_image, roi_coords=[x,y,w,h], n_lines=n_lines, direction=direction)
print(f"Top wall median y-coordinate: {top_wall_y}")
print(f"Bottom wall median y-coordinate: {bottom_wall_y}")

# Update ROI with detected wall y-coordinates
y = top_wall_y + n_pixels
h = bottom_wall_y - top_wall_y - n_pixels*2

# Optical parameters
pixel_size = 5.374e-6
framerate = 200
resolution = 1024

# Initialize an empty dataframe
df_raw = pd.DataFrame(columns=['Image', 'X1', 'Y1', 'X2', 'Y2', 'Xm', 'Ym', 'theta1', 'theta2'])

# Start the loop
for i in range(len(image_files)):
    
    plt.figure(figsize=(7, 7))
    image = cv2.imread(os.path.join(path_images, image_files[i]))
    roi_image = image[y:y+h, x:x+w]
    
    # Detect interface points and fit a polynomial
    outliers, inliers, fitted, coeffs = detect_interface(roi_image, thres=50, poly_degree=poly_degree, residual_thres=15, 
                                                         max_trials=1000, min_samples=1, direction=direction)
    inliers = inliers + [x,y]
    fitted = fitted + [x,y]
    
    # Calculate contact angle
    theta1 = calculate_contact_angle(coeffs, y-y, direction=direction)
    theta2 = calculate_contact_angle(coeffs, h, direction=direction)
    #print(f"The contact angle on top wall is {theta1} and on bottom wall is {theta2}")
    
    # Get the boundary points and middle of the interface
    w1_x, w1_y, w2_x, w2_y, m_x, m_y  = get_interface_points(inliers, y, y+h, n_pixels=2, direction=direction)    
    df_raw = pd.concat([df_raw, pd.DataFrame([{'Image': image_files[i], 
                            'X1': w1_x, 'Y1': w1_y,
                            'X2': w2_x, 'Y2': w2_y,
                            'Xm': m_x, 'Ym': m_y,  
                            'theta1': theta1, 'theta2': theta2}])], ignore_index=True) 
    
    # Draw ROI and fitted polynomial on the image
    processed_image = image.copy()
    cv2.rectangle(processed_image, (x, y), (x+w, y+h), (255, 255, 255), 2)
    processed_image = write_text(processed_image, f'Top: {theta1:.2f}', (700,50)) 
    processed_image = write_text(processed_image, f'Bottom: {theta2:.2f}', (700,100))
    plt.imshow(cv2.cvtColor(processed_image, cv2.COLOR_BGR2RGB))
    plt.plot(fitted[:, 0], fitted[:, 1], 'w--')
    
    # Save processed image with original name
    processed_name = os.path.splitext(image_files[i])[0] + ".jpg"
    #plt.savefig(os.path.join(path_output, processed_name), dpi=300)
    
print("All images postprocessed.")
display(df_raw)

In [ ]:
# Calculate interface movement and dynamic contact angle
df =  cal_interface_movement(df_raw, pixel_size, framerate, resolution)
display(df)

# Save the data to csv
df.to_csv(os.path.join(path_results, title + '.csv'), index=False)

In [ ]:
# Plot the dynamic contact angle with time and save the figure
data_visualization(df, model="contact angle", path_results=path_results, title=title)

In [ ]:
#Plot the interface displacement along x-axis and save the figure
data_visualization(df, model="displacement-x", path_results=path_results, title=title)